In [1]:
pip install opencv-python

Note: you may need to restart the kernel to use updated packages.


In [3]:
from keras.optimizers import RMSprop
from keras.preprocessing.image import ImageDataGenerator
import cv2
from keras.models import Sequential
from keras.layers import Conv2D, Input, ZeroPadding2D, BatchNormalization, Activation, MaxPooling2D, Flatten, Dense,Dropout
from keras.models import Model, load_model
from keras.callbacks import TensorBoard, ModelCheckpoint
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.utils import shuffle
import imutils
import numpy as np

In [4]:
model = Sequential([
    Conv2D(100, (3,3), activation='relu', input_shape=(150, 150, 3)),
    MaxPooling2D(2,2),
    
    Conv2D(100, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    
    Flatten(),
    Dropout(0.5),
    Dense(50, activation='relu'),
    Dense(2, activation='softmax')
])
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])

In [7]:
TRAINING_DIR = "./New Masks Dataset/Train"
train_datagen = ImageDataGenerator(rescale=1.0/255,
                                   rotation_range=40,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   fill_mode='nearest')

train_generator = train_datagen.flow_from_directory(TRAINING_DIR, 
                                                    batch_size=10, 
                                                    target_size=(150, 150))
VALIDATION_DIR = "./New Masks Dataset/Train"
validation_datagen = ImageDataGenerator(rescale=1.0/255)

validation_generator = validation_datagen.flow_from_directory(VALIDATION_DIR, 
                                                         batch_size=10, 
                                                         target_size=(150, 150))

Found 600 images belonging to 2 classes.
Found 600 images belonging to 2 classes.


In [14]:
checkpoint = ModelCheckpoint('model2-{epoch:03d}.h5',monitor='val_loss',verbose=0,save_best_only=True,mode='auto')

In [15]:
history = model.fit_generator(train_generator,
                              epochs=10,
                              validation_data=validation_generator,
                              callbacks=[checkpoint])

Epoch 1/10


<ipython-input-15-6272b23e5a0b>:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(train_generator,


60/60 [==============================] - 26s 439ms/step - loss: 0.3289 - acc: 0.8683 - val_loss: 0.2055 - val_acc: 0.9283
Epoch 2/10
60/60 [==============================] - 25s 420ms/step - loss: 0.3473 - acc: 0.8483 - val_loss: 0.1826 - val_acc: 0.9383
Epoch 3/10
60/60 [==============================] - 27s 445ms/step - loss: 0.3273 - acc: 0.8750 - val_loss: 0.1901 - val_acc: 0.9333
Epoch 4/10
60/60 [==============================] - 25s 409ms/step - loss: 0.3217 - acc: 0.8667 - val_loss: 0.2092 - val_acc: 0.9333
Epoch 5/10
60/60 [==============================] - 26s 439ms/step - loss: 0.3590 - acc: 0.8567 - val_loss: 0.1974 - val_acc: 0.9267
Epoch 6/10
60/60 [==============================] - 24s 397ms/step - loss: 0.3236 - acc: 0.8783 - val_loss: 0.1883 - val_acc: 0.9350
Epoch 7/10
60/60 [==============================] - 24s 397ms/step - loss: 0.3240 - acc: 0.8600 - val_loss: 0.2018 - val_acc: 0.9367
Epoch 8/10
60/60 [==============================] - 24s 397ms/step - loss: 0.307

In [19]:
import cv2
import numpy as np
from keras.models import load_model
model=load_model("./model2-002.h5")

results={0:'without mask',1:'mask'}
GR_dict={0:(0,0,255),1:(0,255,0)}

rect_size = 4
cap = cv2.VideoCapture(0) 


haarcascade = cv2.CascadeClassifier('/home/user_name/.local/lib/python3.6/site-packages/cv2/data/haarcascade_frontalface_default.xml')

while True:
    (rval, im) = cap.read()
    im=cv2.flip(im,1,1) 

    
    rerect_size = cv2.resize(im, (im.shape[1] // rect_size, im.shape[0] // rect_size))
    faces = haarcascade.detectMultiScale(rerect_size)
    for f in faces:
        (x, y, w, h) = [v * rect_size for v in f] 
        
        face_img = im[y:y+h, x:x+w]
        rerect_sized=cv2.resize(face_img,(150,150))
        normalized=rerect_sized/255.0
        reshaped=np.reshape(normalized,(1,150,150,3))
        reshaped = np.vstack([reshaped])
        result=model.predict(reshaped)

        
        label=np.argmax(result,axis=1)[0]
      
        cv2.rectangle(im,(x,y),(x+w,y+h),GR_dict[label],2)
        cv2.rectangle(im,(x,y-40),(x+w,y),GR_dict[label],-1)
        cv2.putText(im, results[label], (x, y-10),cv2.FONT_HERSHEY_SIMPLEX,0.8,(255,255,255),2)

    cv2.imshow('LIVE',   im)
    key = cv2.waitKey(10)
    
    if key == 27: 
        break

cap.release()

cv2.destroyAllWindows()

AttributeError: 'NoneType' object has no attribute 'shape'

In [12]:
import cv2
from tensorflow.keras.models import load_model
from keras.preprocessing.image import load_img,img_to_array
import numpy as np
model=load_model('saved_model-220321-141612.h5')
img_width , img_height = 150,150
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
cap=cv2.VideoCapture('video.mp4')
cap=cv2.VideoCapture(0)
img_count_full=0
font=cv2.FONT_HERSHEY_SIMPLEX
ord=(1,1)
class_label=''
fontScale=1
color=(255,0,0)
thickness=2
while True:
    
    img_count_full +=1
    response , color_img=cap.read()

    if response == False:
        break

        scale = 50
        width = int(color_img.shape[1]*scale/100)
        height = int(color_img.shape[0]*scale/100)
        dim = (width,height)

        color_img = cv2.resize(color_img, dim, interpolation = cv2.INTER_AREA)

        gray_img = cv2.cvtColor(color_img,cv2.COLOR_BGR2GRAY)

        faces = face_cascade.detectMultiScale(gray_img,1.1,6)

        img_count=0
        for(x,y,w,h) in faces:
            org = (x-10,y-10)
            img_count += 1
            color_face = color_img[y:y+h,x:x+w]
            cv2.imwrite('input%d%dface.jpg'%(img_count_full,img_count),color_face)
            img = load_img('input%d%dface.jpg'%(img_count_full,img_count),target_size=(img_widt,img_height))
            img = img_to_array(img)
            img = np.expand_dims(img,axis=0)
            prediction = model.predict(img)

            if prediction == 0:
                class_label = "Mask"
                color = (0,255,0) 

            else:
                                                                                       
                class_label = "No Mask"
                color = (0,0,255) 
            cv2.rectangle(color_img,(x,y),(x+w,y+h),(255,0,0),3) #BLACK RECTANGLE
            cv2.putText(color_img,class_label,org,font,fontScale,color,thickness,cv2.LINE_AA)
                                                                                       
        cv2.imshow('Face mask detection',color_img)

        key=cv2.waitKey(1)

        if key == 113:
            break

cap.release()
cv2.destroyAllWindows()

ImportError: cannot import name 'load_img' from 'keras.preprocessing.image' (/home/aashi/.local/lib/python3.8/site-packages/keras/preprocessing/image.py)